# initialize

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import shutil
import re

In [2]:
# Jobsites:
# data-jobs.ch    integrated
# jobs.ch         integrated
# ostjob.ch       integrated
# glassdoor       not integrated
# jobagent.ch     not integrated
# jobscout24      not integrated

In [3]:
search_terms = ['data%20scientist', 'data%20analyst', 'BI', 'business%20intelligence']
job_list = []

# data-jobs.ch
### no search therms used -> all jobs are loaded

In [4]:
continue_reading = 1
page_number = 1

while continue_reading == 1:
    url = f"https://data-jobs.ch/search?page={page_number}#results"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    jobs = soup.find_all(name="li", class_="item")

    for job in jobs:
        try:
            if 'banner' not in job['class']:
                job_dict = {}
                # job_id = job['data-gtm-jid']
                # job_dict['job_id'] = job_id if job_id else "N/A"
                job_name = job.find(name="span", class_="jobtitle")
                job_dict['job_name'] = job_name.text.strip() if job_name else "N/A"
                job_company = job.find(name="span", class_="company")
                job_dict['job_company'] = job_company.text.strip() if job_company else "N/A"
                
                job_location = job.find(name="span", class_="location")
                job_location = job_location.text.strip() if job_location else "N/A"
                job_location = re.sub(r'^CH-', '', job_location)
                job_location = re.sub(r'^\d{4}', '', job_location)
                job_dict['job_location'] = job_location.strip()
                
                job_dict['job_percentage'] = "N/A"
                job_link = job.find(name="a", class_="title")['href']
                job_dict['job_link'] = job_link if job_link else "N/A"
                job_dict['job_date'] = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
                job_dict['job_source'] = "data-jobs.ch"
                # job_marker = job.find(name="span", class_="marker")
                # job_dict['job_marker'] = job_marker.text.strip() if job_marker else "N/A"

            job_list.append(job_dict)
        except Exception as ex:
            print(f"Error processing jobs: {ex}")

    page_number += 1

    if not soup.find("span", class_="fa-solid fa-angles-right") or page_number > 100:
        continue_reading = 0

print(f"Anzahl gefundene Jobs (Total): {len(job_list)}")
df = pd.DataFrame(job_list)
df.to_csv('data_jobs_ch.csv', index=False)

Anzahl gefundene Jobs (Total): 329


# jobs.ch

In [5]:
for search_term in search_terms:
    continue_reading = 1
    page_number = 1
    while continue_reading == 1:
        # Make a request to the webpage
        url = f"https://www.jobs.ch/de/stellenangebote/?page={page_number}&term={search_term}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        jobs = soup.find_all(name="div", class_="Div-sc-1cpunnt-0", attrs={"data-feat": "searched_jobs"})

        for job in jobs:
            try:
                job_dict = {}
                job_name = job.select_one("article:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(3) > span:nth-child(1)")
                job_dict['job_name'] = job_name.text.strip() if job_name else "N/A"
                job_company = job.select_one("div:nth-child(5) > p:nth-child(1) > strong:nth-child(1)")
                job_dict['job_company'] = job_company.text.strip() if job_company else "N/A"
                
                job_location = job.select_one("div:nth-child(1) > p:nth-child(2)")
                job_location = job_location.text.strip() if job_location else "N/A"
                job_location = re.sub(r'^CH-', '', job_location)
                job_location = re.sub(r'^\d{4}', '', job_location)
                job_dict['job_location'] = job_location.strip()
                
                job_percentage = job.select_one("div:nth-child(4) > div:nth-child(2) > p:nth-child(2)")
                job_dict['job_percentage'] = job_percentage.text.strip() if job_percentage else "N/A"
                job_link = "https://www.jobs.ch" + job.select_one("a:nth-child(1)")['href']
                job_dict['job_link'] = job_link if job_link else "N/A"
                job_dict['job_date'] = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
                job_dict['job_source'] = "jobs.ch"

                job_list.append(job_dict)
            except Exception as ex:
                print(f"Error processing jobs: {ex}")

        page_number += 1

        if not soup.find("a", attrs={"data-cy": "paginator-next"}) or page_number > 100:
            continue_reading = 0

print(f"Anzahl gefundene Jobs (Total): {len(job_list)}")
df = pd.DataFrame(job_list)
df.to_csv('jobs_ch.csv', index=False)

Anzahl gefundene Jobs (Total): 2560


# ost-job.ch

In [6]:
for search_term in search_terms:
    continue_reading = 1
    page_number = 1
    while continue_reading == 1:
        url = f"https://www.ostjob.ch/job/suche-{search_term}-seite-{page_number}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        jobs = soup.find_all(name="div", class_="vacancy-list-card__body")

        for job in jobs:
            try:
                job_dict = {}
                job_name = job.find(name="span", class_="vacancy-list-card__title")
                job_dict['job_name'] = job_name.text.strip() if job_name else "N/A"
                job_company = job.find(name="li", class_="vacancy-list-card__companyname")
                job_dict['job_company'] = job_company.text.strip() if job_company else "N/A"
                
                job_location = job.find(name="li", class_="vacancy-list-card__location")
                job_location = job_location.text.strip() if job_location else "N/A"
                job_location = re.sub(r'^CH-', '', job_location)
                job_location = re.sub(r'^\d{4}', '', job_location)
                job_dict['job_location'] = job_location.strip()
                
                job_percentage = job.find(name="li", class_="vacancy-list-card__type")
                job_dict['job_percentage'] = job_percentage.text.strip() if job_percentage else "N/A"
                job_link = job.find(name="a", class_="vacancy-list-card__link-content")['href']
                job_dict['job_link'] = "https://www.ostjob.ch" + job_link if job_link else "N/A"
                job_dict['job_date'] = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
                job_dict['job_source'] = "ost-job.ch"

                job_list.append(job_dict)
            except Exception as ex:
                print(f"Error processing jobs: {ex}")

        page_number += 1

        if not soup.find("i", class_="oj oj-arrow", attrs={"style": ""}) or page_number > 10:
            continue_reading = 0

print(f"Anzahl gefundene Jobs (Total): {len(job_list)}")
df = pd.DataFrame(job_list)
df.to_csv('ost_jobs_ch.csv', index=False)

Anzahl gefundene Jobs (Total): 2622


# Data cleaning and -merging

In [59]:
df_new_jobs = pd.DataFrame(job_list).reset_index(drop=True)
df_new_jobs['job_list_status'] = "new"

In [60]:
df_old_jobs = pd.read_excel('job_list.xlsx', sheet_name='job_list', usecols='B:H').reset_index(drop=True)

df_old_jobs['job_company'] = df_old_jobs['job_company'].fillna('N/A')

In [61]:
df_plz_ort = pd.read_csv('gwr_plzmap.csv', delimiter=',')
df_plz_ort = df_plz_ort.rename(columns={'plznamk': 'job_location', 'plz4': 'job_plz'})
df_plz_ort = df_plz_ort.sort_values(by='job_plz')
df_plz_ort = df_plz_ort.drop_duplicates(subset='job_location', keep='first')

In [62]:
df_new_jobs = df_new_jobs.merge(df_plz_ort[['job_location', 'job_plz']], on='job_location', how='left')


In [63]:
def convert_to_int_or_na(val):
    if pd.isna(val):
        return 9999
    else:
        return int(val)

In [64]:
df_old_jobs['job_plz'] = df_old_jobs['job_plz'].apply(convert_to_int_or_na)
df_new_jobs['job_plz'] = df_new_jobs['job_plz'].apply(convert_to_int_or_na).astype(int)

In [65]:
df_new_jobs.shape

(2622, 9)

In [66]:
df_new_jobs.dtypes

job_name           object
job_company        object
job_location       object
job_percentage     object
job_link           object
job_date           object
job_source         object
job_list_status    object
job_plz             int64
dtype: object

In [67]:
df_new_jobs = df_new_jobs[((((df_new_jobs['job_plz']) >= 5000) & (df_new_jobs['job_plz'] < 6300)) | (df_new_jobs['job_plz'] >= 8000))]

In [69]:
column_to_move = df_new_jobs.columns[8]
df_new_jobs.insert(1, column_to_move, df_new_jobs.pop(column_to_move))

In [70]:
df_merged = pd.concat([df_old_jobs, df_new_jobs]).sort_values('job_list_status', ascending=False)

df_merged = df_merged.drop_duplicates(subset=['job_name', 'job_company'], keep='first')

In [71]:
df_new_jobs.shape

(1719, 9)

In [72]:
df_old_jobs.shape

(2444, 7)

In [73]:
df_merged.shape

(2408, 9)

In [74]:
df_merged_new = df_merged[df_merged['job_list_status'] == 'new']

In [75]:
df_merged_new.shape

(30, 9)

# save complete list to xlsx-files

In [76]:
# BACKUP !!!

source_old_jobs = 'job_list.xlsx'
destination_old_jobs = f'./backup/job_list_{datetime.now().strftime("%d.%m.%Y %H_%M_%S")}.xlsx'

shutil.copy2(source_old_jobs, destination_old_jobs)


'./backup/job_list_24.08.2023 20_55_36.xlsx'

In [77]:
df_merged_new.to_excel('new_jobs.xlsx', sheet_name='new_jobs')